In [3]:
import torch

# Check if CUDA is available
print(torch.cuda.is_available())

# If True, you can get more details
if torch.cuda.is_available():
    print(f"Number of CUDA devices: {torch.cuda.device_count()}")
    print(f"Current CUDA device: {torch.cuda.current_device()}")
    print(f"CUDA device name: {torch.cuda.get_device_name(0)}")

False


In [5]:
import subprocess
import platform

def check_cuda():
    system = platform.system()
    
    # Check NVIDIA driver
    try:
        if system == "Windows":
            result = subprocess.run(["nvidia-smi"], capture_output=True, text=True)
        elif system in ["Linux", "Darwin"]:
            result = subprocess.run(["nvidia-smi"], capture_output=True, text=True)
        
        print("NVIDIA Driver detected:")
        print(result.stdout)
        return True
    except FileNotFoundError:
        print("NVIDIA drivers not found.")
        return False

# Run the check
check_cuda()

NVIDIA Driver detected:
Sun Dec 15 12:06:02 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.81                 Driver Version: 560.81         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   50C    P8             10W /   85W |      18MiB /   6144MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------

True

In [4]:
print("PyTorch CUDA Available:", torch.cuda.is_available())
print("CUDA Device Count:", torch.cuda.device_count())

PyTorch CUDA Available: False
CUDA Device Count: 0


In [6]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import os

# U-Net Model Architecture
def unet(input_size=(256, 256, 3)):
    inputs = Input(input_size)

    # Contracting Path (Encoder)
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)

    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)

    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)

    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D((2, 2))(conv4)

    # Bottleneck
    conv5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(conv5)

    # Expansive Path (Decoder)
    up6 = UpSampling2D((2, 2))(conv5)
    concat6 = concatenate([up6, conv4])
    conv6 = Conv2D(512, (3, 3), activation='relu', padding='same')(concat6)
    conv6 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv6)

    up7 = UpSampling2D((2, 2))(conv6)
    concat7 = concatenate([up7, conv3])
    conv7 = Conv2D(256, (3, 3), activation='relu', padding='same')(concat7)
    conv7 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv7)

    up8 = UpSampling2D((2, 2))(conv7)
    concat8 = concatenate([up8, conv2])
    conv8 = Conv2D(128, (3, 3), activation='relu', padding='same')(concat8)
    conv8 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv8)

    up9 = UpSampling2D((2, 2))(conv8)
    concat9 = concatenate([up9, conv1])
    conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(concat9)
    conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv9)

    # Output layer for binary classification
    x = Conv2D(1, (1, 1), activation='sigmoid')(conv9)  # Single output per pixel (256x256)

    # Apply Global Average Pooling to reduce to a single scalar output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)  # Converts output from (None, 256, 256, 1) to (None, 1)

    model = Model(inputs, x)
    return model




Epoch 1/10
  16/3152 ━━━━━━━━━━━━━━━━━━━━ 35:31:12 41s/step - accuracy: 0.5321 - loss: 0.6946

In [ ]:
import os
import shutil
import random

def copy_random_images(src_dir, dest_dir, limit=500):
    """

    Args:
        src_dir (str): Вихідна директорія з підпапками (категоріями).
        dest_dir (str): Цільова директорія для обмежених даних.
        limit (int): Кількість файлів для копіювання з кожної підпапки.
    """
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    for class_name in os.listdir(src_dir):
        class_src_path = os.path.join(src_dir, class_name)
        class_dest_path = os.path.join(dest_dir, class_name)

        if not os.path.isdir(class_src_path):
            continue  # Пропустити, якщо це не папка

        if not os.path.exists(class_dest_path):
            os.makedirs(class_dest_path)

        # Отримати всі файли та скопіювати випадкові limit
        files = os.listdir(class_src_path)
        random.shuffle(files)  # Перемішати файли
        selected_files = files[:limit]  # Вибрати перші limit файлів після перемішування

        for file in selected_files:
            src_file_path = os.path.join(class_src_path, file)
            dest_file_path = os.path.join(class_dest_path, file)
            shutil.copy(src_file_path, dest_file_path)

# Використання функції для обмеження тренувального, валідаційного та тестового наборів
copy_random_images('split_data/train', 'limited_data/train', limit=1500)
copy_random_images('split_data/val', 'limited_data/val', limit=1500)
copy_random_images('split_data/test', 'limited_data/test', limit=1500)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'split_data/train/',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',  # Output 0 or 1 for each image
    color_mode='rgb'
)

val_generator = test_datagen.flow_from_directory(
    'split_data/val/',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',  # Output 0 or 1 for each image
    color_mode='rgb'
)

test_generator = test_datagen.flow_from_directory(
    'split_data/test/',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',  # Output 0 or 1 for each image
    color_mode='rgb'
)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
next_train_generator = train_datagen.flow_from_directory(
    'limited_data/train/',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',  # Output 0 or 1 for each image
    color_mode='rgb'
)

next_val_generator = test_datagen.flow_from_directory(
    'limited_data/val/',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',  # Output 0 or 1 for each image
    color_mode='rgb'
)

next_test_generator = test_datagen.flow_from_directory(
    'limited_data/test/',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',  # Output 0 or 1 for each image
    color_mode='rgb'
)




In [ ]:
# Compile the U-Net model
model = unet(input_size=(256, 256, 3))
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])



In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tqdm.keras import TqdmCallback

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model_32batch.keras', monitor='val_loss', save_best_only=True)

In [ ]:
history = model.fit(next_train_generator, epochs=50, validation_data=next_val_generator, callbacks=[ early_stopping, model_checkpoint, TqdmCallback(verbose=1)])

# Plot loss and accuracy curves
plt.figure(figsize=(12, 4))

# Plot Training and Validation Loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.legend()

# Plot Training and Validation Accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.legend()

plt.show()



In [ ]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}, Test loss: {test_loss}")

# Save the model
model.save('cancer_detection_unet_batch32_1500im_lr1e4.h5')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Visualizing predictions
images, labels = next(test_generator)  # Get a batch of images
predictions = model.predict(images)

# Set the number of images per row and calculate the number of rows
images_per_row = 5
num_images = 10  # Number of images to display
num_rows = num_images // images_per_row + (1 if num_images % images_per_row != 0 else 0)

# Create the plot
fig, axes = plt.subplots(num_rows, images_per_row, figsize=(15, num_rows * 3))

# Flatten the axes array to easily iterate through each subplot
axes = axes.flatten()

for i in range(num_images):
    ax = axes[i]  # Get the corresponding axis for this image
    ax.imshow(images[i])  # Display the image
    ax.set_title(f"Pred: {predictions[i]}, True: {labels[i]}")
    ax.axis('off')  # Turn off the axis for cleaner display

# Hide any unused axes (if the number of images is less than the grid size)
for j in range(num_images, len(axes)):
    axes[j].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
#Testing Augmentation

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.35,
    height_shift_range=0.35,
    shear_range=0.35,
    zoom_range=0.35,
    horizontal_flip=True,
    brightness_range = [0.5, 1.5]
    fill_mode='nearest'
)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
next_train_generator = train_datagen.flow_from_directory(
    'limited_data/train/',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',  # Output 0 or 1 for each image
    color_mode='rgb'
)

next_val_generator = test_datagen.flow_from_directory(
    'limited_data/val/',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',  # Output 0 or 1 for each image
    color_mode='rgb'
)

next_test_generator = test_datagen.flow_from_directory(
    'limited_data/test/',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',  # Output 0 or 1 for each image
    color_mode='rgb'
)

In [ ]:
model = unet(input_size=(256, 256, 3))
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tqdm.keras import TqdmCallback

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model_32batch_aug.keras', monitor='val_loss', save_best_only=True)

In [ ]:
history = model.fit(next_train_generator, epochs=50, validation_data=next_val_generator, callbacks=[ early_stopping, model_checkpoint, TqdmCallback(verbose=1)])

# Plot loss and accuracy curves
plt.figure(figsize=(12, 4))

# Plot Training and Validation Loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.legend()

# Plot Training and Validation Accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.legend()

plt.show()

In [ ]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}, Test loss: {test_loss}")

# Save the model
model.save('cancer_detection_unet_batch32_1500im_lr1e4_aug.h5')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Visualizing predictions
images, labels = next(test_generator)  # Get a batch of images
predictions = model.predict(images)

# Set the number of images per row and calculate the number of rows
images_per_row = 5
num_images = 10  # Number of images to display
num_rows = num_images // images_per_row + (1 if num_images % images_per_row != 0 else 0)

# Create the plot
fig, axes = plt.subplots(num_rows, images_per_row, figsize=(15, num_rows * 3))

# Flatten the axes array to easily iterate through each subplot
axes = axes.flatten()

for i in range(num_images):
    ax = axes[i]  # Get the corresponding axis for this image
    ax.imshow(images[i])  # Display the image
    ax.set_title(f"Pred: {predictions[i]}, True: {labels[i]}")
    ax.axis('off')  # Turn off the axis for cleaner display

# Hide any unused axes (if the number of images is less than the grid size)
for j in range(num_images, len(axes)):
    axes[j].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    #brightness_range = [0.5, 1.5]
    fill_mode='nearest'
)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
next_train_generator = train_datagen.flow_from_directory(
    'limited_data/train/',
    target_size=(256, 256),
    batch_size=16,
    class_mode='binary',  # Output 0 or 1 for each image
    color_mode='rgb'
)

next_val_generator = test_datagen.flow_from_directory(
    'limited_data/val/',
    target_size=(256, 256),
    batch_size=16,
    class_mode='binary',  # Output 0 or 1 for each image
    color_mode='rgb'
)

next_test_generator = test_datagen.flow_from_directory(
    'limited_data/test/',
    target_size=(256, 256),
    batch_size=16,
    class_mode='binary',  # Output 0 or 1 for each image
    color_mode='rgb'
)

In [ ]:
model = unet(input_size=(256, 256, 3))
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(next_train_generator, epochs=50, validation_data=next_val_generator, callbacks=[ early_stopping, model_checkpoint, TqdmCallback(verbose=1)])

# Plot loss and accuracy curves
plt.figure(figsize=(12, 4))

# Plot Training and Validation Loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.legend()

# Plot Training and Validation Accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.legend()

plt.show()

In [ ]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}, Test loss: {test_loss}")

# Save the model
model.save('cancer_detection_unet_batch16_1500im_lr1e4.h5')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Visualizing predictions
images, labels = next(test_generator)  # Get a batch of images
predictions = model.predict(images)

# Set the number of images per row and calculate the number of rows
images_per_row = 5
num_images = 10  # Number of images to display
num_rows = num_images // images_per_row + (1 if num_images % images_per_row != 0 else 0)

# Create the plot
fig, axes = plt.subplots(num_rows, images_per_row, figsize=(15, num_rows * 3))

# Flatten the axes array to easily iterate through each subplot
axes = axes.flatten()

for i in range(num_images):
    ax = axes[i]  # Get the corresponding axis for this image
    ax.imshow(images[i])  # Display the image
    ax.set_title(f"Pred: {predictions[i]}, True: {labels[i]}")
    ax.axis('off')  # Turn off the axis for cleaner display

# Hide any unused axes (if the number of images is less than the grid size)
for j in range(num_images, len(axes)):
    axes[j].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range = [0.8, 1.2]
    fill_mode='nearest'
)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
next_train_generator = train_datagen.flow_from_directory(
    'limited_data/train/',
    target_size=(256, 256),
    batch_size=16,
    class_mode='binary',  # Output 0 or 1 for each image
    color_mode='rgb'
)

next_val_generator = test_datagen.flow_from_directory(
    'limited_data/val/',
    target_size=(256, 256),
    batch_size=16,
    class_mode='binary',  # Output 0 or 1 for each image
    color_mode='rgb'
)

next_test_generator = test_datagen.flow_from_directory(
    'limited_data/test/',
    target_size=(256, 256),
    batch_size=16,
    class_mode='binary',  # Output 0 or 1 for each image
    color_mode='rgb'
)

In [ ]:
model = unet(input_size=(256, 256, 3))
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(next_train_generator, epochs=50, validation_data=next_val_generator, callbacks=[ early_stopping, model_checkpoint, TqdmCallback(verbose=1)])

# Plot loss and accuracy curves
plt.figure(figsize=(12, 4))

# Plot Training and Validation Loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.legend()

# Plot Training and Validation Accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.legend()

plt.show()

In [ ]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}, Test loss: {test_loss}")

# Save the model
model.save('cancer_detection_unet_batch16_1500im_lr1e4_aug.h5')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Visualizing predictions
images, labels = next(test_generator)  # Get a batch of images
predictions = model.predict(images)

# Set the number of images per row and calculate the number of rows
images_per_row = 5
num_images = 10  # Number of images to display
num_rows = num_images // images_per_row + (1 if num_images % images_per_row != 0 else 0)

# Create the plot
fig, axes = plt.subplots(num_rows, images_per_row, figsize=(15, num_rows * 3))

# Flatten the axes array to easily iterate through each subplot
axes = axes.flatten()

for i in range(num_images):
    ax = axes[i]  # Get the corresponding axis for this image
    ax.imshow(images[i])  # Display the image
    ax.set_title(f"Pred: {predictions[i]}, True: {labels[i]}")
    ax.axis('off')  # Turn off the axis for cleaner display

# Hide any unused axes (if the number of images is less than the grid size)
for j in range(num_images, len(axes)):
    axes[j].axis('off')

plt.tight_layout()
plt.show()